In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import os
import numpy
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [ ]:
path = "/content/drive/MyDrive/Data"
files = os.listdir(path)
orb = cv2.ORB_create()
#sift = cv2.xfeatures2d.SIFT_create()
X=numpy.zeros(shape=(1,500*32))
Y=numpy.zeros(shape=(1,))
count=-1
for f in files:
      if not (f.startswith('.')):
        folder = os.path.join(path, f)
        filenames = os.listdir(folder)
        count+=1
        for fi in filenames:
            if not (fi.startswith('.')):
                filepath = os.path.join(folder, fi)
                #print filepath
                image = cv2.imread(filepath, 0)
                kp, features = orb.detectAndCompute(image,None)
                #print features.shape
                features=features.reshape(1,500*32)
                #print features.shape
                X = numpy.vstack((X, features))
                Y=numpy.vstack((Y,numpy.asarray([count])))
#print X.shape
Y=Y.reshape(Y.shape[0],)

In [ ]:
print("printing data shape {}".format(X.shape))

# Splitting the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Training the Random Forest Classifier
K = 1000
clf1 = RandomForestClassifier(n_estimators=K)
clf1.fit(X_train, Y_train)

# Evaluating the model
y_pred = clf1.predict(X_test)
print("Accuracy on test set: {}".format(accuracy_score(Y_test, y_pred)))
# Predicting for custom input file

printing data shape (548, 16000)


KeyboardInterrupt: 

In [ ]:
filename = "model_RandomForest.pickle"

In [ ]:
pickle.dump(clf1, open(filename, "wb"))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
C_score = []
grid = numpy.arange(0.01, 3, 0.1)
for K in grid:
    clf2 = SVC(C=K)
    # Calculate the mean scores for each value of hyperparameter C
    scores = cross_val_score(clf2, X_train, Y_train, cv=5)
    print(scores.mean())
    C_score.append(scores.mean())

# Display the maximum score achieved at which hyperparameter value
print (" max score is ", max(C_score), " at C = ", grid[C_score.index(max(C_score))])
clf2 = SVC(C=grid[C_score.index(max(C_score))])
clf2.fit(X_train, Y_train)

0.2206960385042577
0.2206960385042577
0.2206960385042577
0.2206960385042577
0.2206960385042577
0.2206960385042577
0.22339874120696038
0.22613846723435765
0.22613846723435765
0.23706034801925213
0.2589041095890411
0.29163272861903
0.30529433543132173
0.31884487226952984
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
0.32428730099962977
 max score is  0.32428730099962977  at C =  1.4100000000000001


SVC(C=1.4100000000000001)

In [ ]:
filename = "model_SVM.pickle"

In [ ]:
pickle.dump(clf2, open(filename, "wb"))

In [ ]:
!pip install keras


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data directories
train_data_dir = '/content/drive/MyDrive/Data1/train'
validation_data_dir = '/content/drive/MyDrive/Data1/valid'

# Define image preprocessing parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size and image dimensions
batch_size = 32
img_height = 64
img_width = 64

# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Load and preprocess validation data
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 380 images belonging to 5 classes.
Found 108 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Define the CNN model
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding="same",
                 input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(units=5, activation='softmax'))  # Adjusted units for 5 classes

model.add(BatchNormalization(axis=1))
model.add(Dropout(0.25))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 1000)              3

In [ ]:
epochs = 100
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

Epoch 1/100
11/11 [==============================] - 109s 10s/step - loss: 8.3727 - accuracy: 0.1466 - val_loss: 8.7952 - val_accuracy: 0.1875
Epoch 2/100
11/11 [==============================] - 9s 836ms/step - loss: 9.6154 - accuracy: 0.1925 - val_loss: 8.3656 - val_accuracy: 0.1875
Epoch 3/100
11/11 [==============================] - 7s 641ms/step - loss: nan - accuracy: 0.2270 - val_loss: 8.2764 - val_accuracy: 0.1979
Epoch 4/100
11/11 [==============================] - 8s 718ms/step - loss: 10.4877 - accuracy: 0.1724 - val_loss: 7.9586 - val_accuracy: 0.1979
Epoch 5/100
11/11 [==============================] - 8s 674ms/step - loss: 9.2237 - accuracy: 0.1839 - val_loss: 7.6781 - val_accuracy: 0.2396
Epoch 6/100
11/11 [==============================] - 8s 684ms/step - loss: nan - accuracy: 0.2098 - val_loss: 7.2459 - val_accuracy: 0.2396
Epoch 7/100
11/11 [==============================] - 9s 842ms/step - loss: 9.2696 - accuracy: 0.2213 - val_loss: 7.4151 - val_accuracy: 0.2500
Epoc